## implementation of chatbot using NLP

In [1]:
pip install nltk scikit-learn streamlit


Note: you may need to restart the kernel to use updated packages.


In [2]:
import nltk
import random
import os
import ssl
import streamlit as st
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression


In [3]:
ssl._create_default_https_context=ssl._create_unverified_context
nltk.data.path.append(os.path.abspath('nltk_data'))
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
intents = [
    {
        "tag": "greeting",
        "patterns": ["Hi", "Hello", "Hey", "How are you", "What's up"],
        "responses": ["Hi there", "Hello", "Hey", "I'm fine, thank you", "Nothing much"]
    },
    {
        "tag": "goodbye",
        "patterns": ["Bye", "See you later", "Goodbye", "Take care"],
        "responses": ["Goodbye", "See you later", "Take care"]
    },
    {
        "tag": "thanks",
        "patterns": ["Thank you", "Thanks", "Thanks a lot", "I appreciate it"],
        "responses": ["You're welcome", "No problem", "Glad I could help"]
    },
    {
        "tag": "about",
        "patterns": ["What can you do", "Who are you", "What are you", "What is your purpose"],
        "responses": ["I am a chatbot", "My purpose is to assist you", "I can answer questions and provide assistance"]
    },
    {
        "tag": "help",
        "patterns": ["Help", "I need help", "Can you help me", "What should I do"],
        "responses": ["Sure, what do you need help with?", "I'm here to help. What's the problem?", "How can I assist you?"]
    },
    {
        "tag": "age",
        "patterns": ["How old are you", "What's your age"],
        "responses": ["I don't have an age. I'm a chatbot.", "I was just born in the digital world.", "Age is just a number for me."]
    },
    {
        "tag": "weather",
        "patterns": ["What's the weather like", "How's the weather today"],
        "responses": ["I'm sorry, I cannot provide real-time weather information.", "You can check the weather on a weather app or website."]
    },
    {
        "tag": "budget",
        "patterns": ["How can I make a budget", "What's a good budgeting strategy", "How do I create a budget"],
        "responses": ["To make a budget, start by tracking your income and expenses. Then, allocate your income towards essential expenses like rent, food, and bills. Next, allocate some of your income towards savings and debt repayment. Finally, allocate the remainder of your income towards discretionary expenses like entertainment and hobbies.", "A good budgeting strategy is to use the 50/30/20 rule. This means allocating 50% of your income towards essential expenses, 30% towards discretionary expenses, and 20% towards savings and debt repayment.", "To create a budget, start by setting financial goals for yourself. Then, track your income and expenses for a few months to get a sense of where your money is going. Next, create a budget by allocating your income towards essential expenses, savings and debt repayment, and discretionary expenses."]
    },
    {
        "tag": "credit_score",
        "patterns": ["What is a credit score", "How do I check my credit score", "How can I improve my credit score"],
        "responses": ["A credit score is a number that represents your creditworthiness. It is based on your credit history and is used by lenders to determine whether or not to lend you money. The higher your credit score, the more likely you are to be approved for credit.", "You can check your credit score for free on several websites such as Credit Karma and Credit Sesame."]
    },
    {
        "tag": "hobbies",
        "patterns": ["What are your hobbies", "Do you have any hobbies", "What do you like to do for fun"],
        "responses": ["As an AI, I don't have hobbies, but I love helping people and learning new things.", "I don't have hobbies, but I enjoy assisting you with your questions.", "My hobby is providing useful information and support."]
    },
    {
        "tag": "food",
        "patterns": ["What's your favorite food", "Do you like to eat", "What's the best food"],
        "responses": ["I don't eat, but I've heard that pizza is quite popular!", "I don't have a favorite food, but I can help you find a recipe.", "Food preferences vary, but many people love a good burger."]
    },
    {
        "tag": "sports",
        "patterns": ["Do you play any sports", "What's your favorite sport", "Do you like sports"],
        "responses": ["I don't play sports, but I can give you information about them.", "I don't have a favorite sport, but I know a lot about many sports.", "I'm here to help you with any sports-related questions you have."]
    },
    {
        "tag": "movies",
        "patterns": ["What's your favorite movie", "Do you like movies", "What movie should I watch"],
        "responses": ["I don't watch movies, but I can recommend some based on popular opinions.", "Movies are a great way to relax! How about a classic like 'The Shawshank Redemption'?", "If you like action, try 'Mad Max: Fury Road'. For something light-hearted, 'The Grand Budapest Hotel' is a good pick."]
    },
    {
        "tag": "music",
        "patterns": ["What's your favorite music", "Do you like music", "Recommend me some music"],
        "responses": ["I don't listen to music, but I know people love genres like rock, pop, and classical.", "If you're looking for something new, try exploring different genres on a music streaming service.", "How about checking out some top charts to see what's popular these days?"]
    },
    {
        "tag": "travel",
        "patterns": ["Where should I travel", "Recommend a travel destination", "What's a good vacation spot"],
        "responses": ["If you love history, consider visiting Rome or Athens.", "For a tropical getaway, the Maldives or Bali are beautiful destinations.", "If you enjoy city life, New York or Tokyo could be exciting places to explore."]
    },
    {
        "tag": "technology",
        "patterns": ["What's the latest in technology", "Tell me about new tech trends", "What's trending in tech"],
        "responses": ["Artificial Intelligence and Machine Learning are revolutionizing many industries.", "The latest smartphones are incorporating more advanced cameras and AI features.", "Wearable technology, such as smartwatches, is becoming increasingly popular."]
    },
    {
        "tag": "books",
        "patterns": ["What's your favorite book", "Do you read books", "Recommend a good book"],
        "responses": ["I don't read books, but 'To Kill a Mockingbird' by Harper Lee is a classic.", "If you enjoy fiction, '1984' by George Orwell is a must-read.", "For fantasy lovers, 'Harry Potter' series by J.K. Rowling is highly recommended."]
    },
    {
        "tag": "health",
        "patterns": ["How can I stay healthy", "What's a good exercise routine", "Any tips for a healthy diet"],
        "responses": ["Staying healthy involves regular exercise, a balanced diet, and plenty of sleep.", "A good exercise routine includes a mix of cardio, strength training, and flexibility exercises.", "For a healthy diet, focus on eating a variety of fruits, vegetables, lean proteins, and whole grains."]
    },
    {
        "tag": "fitness",
        "patterns": ["How can I get fit", "What's a good fitness plan", "Any tips for staying fit"],
        "responses": ["Getting fit involves regular exercise and a healthy diet.", "A good fitness plan includes a mix of cardio, strength training, and flexibility exercises.", "Staying fit requires consistency and making healthy lifestyle choices."]
    },
    {
        "tag": "motivation",
        "patterns": ["How can I stay motivated", "Any tips for motivation", "How to keep motivation high"],
        "responses": ["Set clear goals and break them into smaller, manageable tasks.", "Find a source of inspiration, like a mentor or a motivational book.", "Reward yourself for achieving milestones to keep your motivation high."]
    },
    {
        "tag": "education",
        "patterns": ["What's a good way to study", "How can I improve my learning", "Any tips for effective studying"],
        "responses": ["A good way to study is by breaking your study sessions into manageable chunks and taking regular breaks.", "Improving your learning involves active engagement, like summarizing information and teaching it to others.", "Effective studying includes creating a study schedule, minimizing distractions, and using different study methods."]
    },
    {
        "tag": "programming",
        "patterns": ["What's a good programming language to learn", "How can I start programming", "Any tips for learning to code"],
        "responses": ["Python is a great programming language for beginners due to its simplicity and versatility.", "Start programming by learning the basics and working on small projects.", "To learn to code, practice regularly, build projects, and seek help from online communities and resources."]
    }
]


In [5]:
vectorizer=TfidfVectorizer()
clf=LogisticRegression(random_state=0,max_iter=10000)

tags=[]
patterns=[]
for intent in intents:
    for pattern in intent['patterns']:
        tags.append(intent['tag'])
        patterns.append(pattern)


In [6]:
x=vectorizer.fit_transform(patterns)
y=tags
clf.fit(x,y)

LogisticRegression(max_iter=10000, random_state=0)

In [7]:
def chat(input_text):
    input_text=vectorizer.transform([input_text])
    tag=clf.predict(input_text)[0]
    for intent in intents:
        if intent['tag']==tag:
            responses=random.choice(intent['responses'])
            return responses

In [8]:
user_input='age'
responses=chat(user_input)
print(responses)

Hi there


In [9]:
user_input='programming'
responses=chat(user_input)
print(responses)

To learn to code, practice regularly, build projects, and seek help from online communities and resources.


In [10]:
user_input='health'
responses=chat(user_input)
print(responses)

Hey


In [11]:
user_input='hi'
responses=chat(user_input)
print(responses)

Hello


In [12]:
user_input='how can i stay healthly?'
responses=chat(user_input)
print(responses)

Reward yourself for achieving milestones to keep your motivation high.


In [13]:
user_input='budget'
responses=chat(user_input)
print(responses)

A good budgeting strategy is to use the 50/30/20 rule. This means allocating 50% of your income towards essential expenses, 30% towards discretionary expenses, and 20% towards savings and debt repayment.


In [14]:
user_input='programming'
responses=chat(user_input)
print(responses)

Python is a great programming language for beginners due to its simplicity and versatility.


In [15]:
user_input='age'
responses=chat(user_input)
print(responses)

Nothing much


In [16]:
user_input='hi'
responses=chat(user_input)
print(responses)

Hey


In [17]:
user_input='motivation'
responses=chat(user_input)
print(responses)

Reward yourself for achieving milestones to keep your motivation high.
